In [2]:
%pip install transformers torch datasets

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/21/02/ae8e595f45b6c8edee07913892b3b41f5f5f273962ad98851dc6a564bbb9/transformers-4.31.0-py3-none-any.whl.metadata
  Using cached transformers-4.31.0-py3-none-any.whl.metadata (116 kB)
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/66/f8/38298237d18d4b6a8ee5dfe390e97bed5adb8e01ec6f9680c0ddf3066728/datasets-2.14.4-py3-none-any.whl.metadata
  Using cached datasets-2.14.4-py3-none-any.whl.metadata (19 kB)
  Obtaining dependency information for huggingface-hub<1.0,>=0.14.1 from https://files.pythonhosted.org/packages/7f/c4/adcbe9a696c135578cabcbdd7331332daad4d49b7c43688bc2d36b3a47d2/huggingface_hub-0.16.4-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.16.4-py3-none-any.whl.metadata (12 kB)
  Using cached tokenizers-0.13.3-cp39-cp39-macosx_10_11_x86_64.whl (4.0 MB

In [4]:
from datasets import load_dataset

data = load_dataset(
    "jamescalam/image-text-demo",
    split="train"
)
# get dataset as a pandas dataframe
df = data.to_pandas()
df.head()

Generating train split: 0 examples [00:00, ? examples/s]

,text,image
0,aerial shot of futuristic city with large moto...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
1,aerial shot of modern city at sunrise,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
2,butterfly landing on the nose of a cat,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
3,cute kitten walking through long grass,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
4,fluffy dog sticking out tongue with yellow bac...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...


In [5]:
type(df['image'])

pandas.core.series.Series

In [8]:
from transformers import CLIPProcessor, CLIPModel
import torch

model_id = "openai/clip-vit-base-patch32"

processor = CLIPProcessor.from_pretrained(model_id)
model = CLIPModel.from_pretrained(model_id)

# move model to device if possible
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model.to(device)

ModuleNotFoundError: No module named 'transformers'

In [5]:
text = data['text']  # 21

tokens = processor(
    text=text,
    padding=True,
    images=None,
    return_tensors='pt'
).to(device)
tokens.keys()

dict_keys(['input_ids', 'attention_mask'])

In [6]:
text_emb = model.get_text_features(
    **tokens
)

In [6]:
print(text_emb.shape)
print(text_emb.min(), text_emb.max())

torch.Size([21, 512])
tensor(-1.1893, grad_fn=<MinBackward1>) tensor(4.8015, grad_fn=<MaxBackward1>)


In [7]:
# IF using dot product similarity, must normalize vectors like so...
import numpy as np

# detach text emb from graph, move to CPU, and convert to numpy array
text_emb = text_emb.detach().cpu().numpy()

# calculate value to normalize each vector by
norm_factor = np.linalg.norm(text_emb, axis=1)
norm_factor.shape

(21,)

In [8]:
text_emb = text_emb.T / norm_factor
# transpose back to (21, 512)
text_emb = text_emb.T
print(text_emb.shape)
print(text_emb.min(), text_emb.max())

(21, 512)
-0.15268454 0.5344989


In [4]:
# from data Dataset object get the image data
df['image']


0     {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
1     {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
2     {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
3     {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
4     {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
5     {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
6     {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
7     {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
8     {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
9     {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
10    {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
11    {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
12    {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
13    {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
14    {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
15    {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
16    {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
17    {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x

In [2]:
image_batch = df['image']

images = processor(
    text=None,
    images=image_batch,
    return_tensors='pt'
)['pixel_values'].to(device)
images.shape

NameError: name 'processor' is not defined